In [3]:
#Импорт библиотек, считывание данных, первые 5 строк из тренировочной выборки
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
train = pd.read_csv('flight_delays_train.csv')
test = pd.read_csv('flight_delays_test.csv')
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [2]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


In [4]:
#Выделение тренировочной, тестовой и отложенной выборок
X_train, y_train = (
    train[["Distance", "DepTime"]].values,
    train["dep_delayed_15min"].map({"Y": 1, "N": 0}).values,
)
X_test = test[["Distance", "DepTime"]].values
X_train_part,X_valid,y_train_part,y_valid = train_test_split(X_train,y_train,test_size=0.3,random_state=42)


In [60]:
X_train_part.shape

(70000, 2)

In [61]:
y_train_part.shape

(70000,)

In [5]:
train["path"]=train['Origin'].str.cat(train['Dest'], sep='-')
print(train["path"])

0        ATL-DFW
1        PIT-MCO
2        RDU-CLE
3        DEN-MEM
4        MDW-OMA
          ...   
99995    SFO-RDD
99996    EWR-DAB
99997    DTW-IAH
99998    DFW-GGG
99999    SEA-SMF
Name: path, Length: 100000, dtype: object


In [13]:
scaler = StandardScaler()
X_train_part = scaler.fit_transform(X_train_part)
X_valid = scaler.transform(X_valid)

In [14]:
from sklearn.model_selection import cross_val_score, cross_val_predict
logit = LogisticRegression(random_state=5,class_weight="balanced")
logit.fit(X_train_part,y_train_part)
parameters = {"C":(0.0001, 0.001, 0.01, 0.1, 1, 10)}
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
logit_grid = GridSearchCV(logit,parameters,cv=skf,verbose=1,n_jobs=-1,scoring="roc_auc")
logit_grid.fit(X_train_part,y_train_part)
y_score = cross_val_score(logit_grid,X_train_part,y_train_part,cv=skf)
y_predict = cross_val_predict(logit_grid,X_train_part,y_train_part,cv=skf)
print(y_predict)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits
[0 0 1 ... 1 0 0]


In [15]:
#Точность модели логистической регрессии
y_pred_logit = logit_grid.predict(X_valid)
print(accuracy_score(y_valid,y_pred_logit))

0.6235666666666667


In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
rf = RandomForestClassifier(
n_estimators=100,
n_jobs=-1,
random_state=42,
oob_score=True,
class_weight="balanced")
rf.fit(X_train_part,y_train_part)
parameters = {"max_features": [1, 2, 4],
    "min_samples_leaf": [3, 5, 7, 9],
    "max_depth": [5, 10, 15],}
#k-fold валидация
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
rf_grid = GridSearchCV(rf, parameters, cv=skf, verbose=1,n_jobs=-1, scoring="roc_auc")
rf_grid.fit(X_train_part,y_train_part)
y_rf_predict = cross_val_predict(rf_grid,X_train_part,y_train_part,cv=skf)
y_rf_predict

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Fitting 5 folds for each of 36 candidates, totalling 180 fits
Fitting 5 folds for each of 36 candidates, totalling 180 fits
Fitting 5 folds for each of 36 candidates, totalling 180 fits
Fitting 5 folds for each of 36 candidates, totalling 180 fits
Fitting 5 folds for each of 36 candidates, totalling 180 fits


array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [17]:
#Точность модели случайного леса
from sklearn.metrics import accuracy_score
y_pred_rf = rf_grid.predict(X_valid)
print(accuracy_score(y_valid,y_pred_rf))

0.6153666666666666


In [18]:
#Градиентный бустинг
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
model = XGBClassifier()
param_grid = {'learning_rate': [0.1, 0.01], 'max_depth': [3, 5, 7]}
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train_part, y_train_part)
y_pred = grid_search.predict(X_valid)
accuracy = accuracy_score(y_valid,y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8154


In [19]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
cross_val_predict(grid_search,X_train_part,y_train_part,cv=skf)

array([0, 0, 0, ..., 0, 0, 0])